# Assignment 2

Import packages

In [ ]:
#%matplotlib tk
% matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd

Load teapot point cloud coordinates

In [ ]:
# Load from text file
tp= np.loadtxt('teapot.txt')
# Number of points in the cloud
n_points = np.shape(tp)
# Transpose and add a fourth coordinate with unitary value 
# (homogeneous coordinates)
tp = np.hstack((tp,np.ones((n_points[0],1),dtype=np.float))).T
# Save it in a data frame
df = pd.DataFrame({"x" : tp[0,:], "y" :tp[1,:], "z" : tp[2,:], 
                                                "w": tp[3,:]})

Transformation matrices in homogeneous coordinates

In [ ]:
# Scaling matrix
def scaling_matrix(s):
    # Scales are given in numpy array s
    # s=np.array([s_x,s_y,s_z])
    
    # Write the scaling matrix here
    S = np.identity(4)
    S[:3, :3] = np.diag(s)
    
    return S

# Translation matrix
def translation_matrix(t):
    # Translation vector is given in numpy array t
    # t=np.array([t_x,t_y,t_z])
    
    # Write the translation matrix here
    T = np.identity(4)
    T[:3, 3] = t
    
    return T

# Rotation matrix
def rotation_matrix(theta, v):
    # Axis for rotation (unit norm vector) is given 
    # is given in numpy array v
    # v=np.array([v_x,v_y,v_z])
    # Rotation angle is given in theta
    
    # Write the rotation matrix here
    st = np.sin(theta)
    ct = np.cos(theta)
    o_ct = 1-ct
    vx2= v[0]**2
    vy2= v[1]**2
    vz2= v[2]**2
    vxy = v[0]*v[1]
    vxz = v[0]*v[2]
    vyz= v[1]*v[2]
    # Rotation
    R = [[ct+vx2*o_ct, vxy*o_ct-v[2]*st, vxz*o_ct+v[1]*st],
         [vxy*o_ct+v[2]*st, ct+vy2*o_ct, vyz*o_ct-v[0]*st],
         [vxz*o_ct-v[1]*st, vyz*o_ct+v[0]*st, ct+vz2*o_ct]]
    R_v = np.identity(4)
    R_v[:3, :3] = R
    
    return R_v

Generation of the animation using the transformations
- To save animation in a movie file uncomment the lines just befor plt.show()

In [ ]:
# Update of the scatter plot after transformation
def update_graph(num):
    # Load point cloud from data frame
    tp=np.vstack((df['x'],df['y'],df['z'],df['w']))
    angle = num*0.01
    # Create uniform scaling matrix 
    M = scaling_matrix([1.0,1.0,1.0])
    # Compose M with rotation matrix
    M = np.dot(rotation_matrix(angle, np.array([0.0,0.0,1.0])),M)
    # Compose M with translation matrix
    M = np.dot(translation_matrix(np.array([6.0,0.0,0.0])),M)
    # Apply composed transformation to the point cloud
    tp_r = np.dot(M,tp)
    # Update scatter plot
    graph._offsets3d = (tp_r[0,:], tp_r[1,:], tp_r[2,:])
    return graph

# 3D figure generation
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
title = ax.set_title('Teapot')

# 3D scatter plot
graph = ax.scatter(tp[0,:], tp[1,:], tp[2,:])
# # Change axis limits if necessary
#ax.set_xlim3d(-8, 8)
#ax.set_ylim3d(-8, 8)
#ax.set_zlim3d(-1, 8)

# Animation function
ani = matplotlib.animation.FuncAnimation(fig, update_graph,1000, 
                                            interval=1, blit=False)

# # Set up formatting for the movie files
# Writer = matplotlib.animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
# # Save animation
# ani.save('teapot.mp4', writer=writer)

plt.show()